# Classification - CNN
Utilizzeremo le **Convolutional Neural Network** (CNN) per la Classificazione di Animali Domestici.

Utilizzeremo come *feature* i *pixel*, ovvero le immagini *raw* in formato RGB.

Faremo uso del framework **TensorFlow** nella sua versione 2.x sfruttando l'API di alto livello **Keras** che semplifica la definizione di CNN profonde e rende disponibili numerosi modelli di reti pre-addestrate su *ImageNet*.

In [ ]:
%matplotlib inline
import time, os, sys
import math
import numpy as np
import matplotlib.pyplot as plt
from joblib import Memory
from sklearn.model_selection import train_test_split
from pathlib import Path
from functools import partial

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Nasconde messaggi di debug
os.environ["CUDA_VISIBLE_DEVICES"] = '0'  # Rende visibile solo la GPU 0
import tensorflow as tf
print('Tensorflow Version', tf.__version__)

import ml_visualization
import ml_utilities

Tensorflow Version 2.7.0


# Dataset

In questa esercitazione useremo un dataset composto di immagini di cani e gatti.

Per caricare il dataset è necessario impostare:
- il percorso in cui sono memorizzate le immagini del dataset (**db_path**);
- il percorso di una cartella di lavoro (**exp_path**) dove, durante l'elaborazione, saranno memorizzati i file intermedi;
- il percorso di una cartella (**checkpoints_path**) dove saranno memorizzati i modelli addestrati;
- il percorso del file contenente la lista delle immagini e relative etichette di classe da utilizzare come *training set* (**train_filelist**);
- il percorso del file contenente la lista delle immagini e relative etichette di classe da utilizzare come *test set* (**test_filelist**);
- il percorso del file contenente la lista dei nomi delle classi (**label_filelist**).

Di seguito sono rese disponibili le tre configurazioni utilizzate durante l'esercitazione.

In [ ]:
db_name = 'ml_bin' #'ml_bin' 'ml_multi'

if db_name == 'ml_bin':
    db_path = 'DBs/CaniGatti_ML18_Es8_2020' 
    train_filelist = 'BinaryTrainingSet.txt'
    test_filelist = 'BinaryTestSet.txt'
    label_filelist = db_path + '/BinaryLabels.txt'
    class_num = 2   # caso 2 classi
    validation_size = 200
    test_has_labels = True
elif db_name == 'ml_multi':
    db_path = 'DBs/CaniGatti_ML18_Es8_2020'
    train_filelist = 'MulticlassTrainingSet.txt'
    test_filelist = 'MulticlassTestSet.txt'
    label_filelist = db_path + '/MulticlassLabels.txt'
    class_num = 12   # caso 12 classi - singoli animali
    validation_size = 200
    test_has_labels = False
else:
    raise ValueError('Database non valido.')

checkpoints_path = './saved_models'
exp_path = './exp_cache'

# Predisposizione di un'area di caching su disco che velocizza la riesecuzione di chiamate di funzioni con gli stessi parametri
memory = Memory(exp_path, verbose=0)  

# Caricamento delle immagini
print('Caricamento in corso ...')
start = time.time()
raw_images_x, label_y = ml_utilities.load_labeled_dataset(train_filelist, db_path, cache=memory)

# Carica le etichette di classe
label_names = [line.rstrip('\n') for line in open(label_filelist,'r')]

print('Caricate %d immagini in %.2f s.' % (len(raw_images_x), time.time() - start))
for i in range(len(label_names)):
    print('{}: {}'.format(label_names[i], np.count_nonzero(label_y == i)))

Caricamento in corso ...
Caricate 650 immagini in 0.13 s.
Gatto: 367
Cane: 283


# Mobile Net

Il modello di CNN (o meglio la famiglia di modelli di CNN) che utilizzeremo in questa esercitazione è noto come [**MobileNet V1**](https://www.tensorflow.org/api_docs/python/tf/keras/applications/MobileNet). Si tratta di un'architettura proposta da Google che garantisce performance di tutto rispetto (equiparabili a una GoogleNet) a un ragionevole costo computazionale, il che la rende particolarmente idonea ad operare in *inference* anche su architetture leggere (prive di GPU), come smartphone Android o iOS.

A seconda del budget computazionale a disposizione, al modello base si può applicare:
- un *resolution multiplier* che determina una riduzione sulla dimensione dell'immagine di input. Default = 224. Valori possibili: [224, 192, 160, 128] 
- un *depth multiplier* che opera una riduzione del numero di feature maps (canali) nei diversi layer, pari a una frazione di quelli del modello base. Default = 1. Valori possibili [1, 0.75, 0.50. 0.25]

Per le piattaforme embedded sono disponibili anche versioni con quantizzazione *eight-bit fixed point* che riduce 4 volte l'occupazione di memoria rispetto a floting point 32 bit.

*Keras* mette a disposizione funzioni semplificate per la creazione del grafo di MobileNet dati i moltiplicatori (resolution e depth). L'immagine di seguito mostra l'architettura della MobileNet V1 con depth e resolution multipliers impostati ai valori di default (1.0 e 224 rispettivamente).

# Fine Tuning vs Addestramento da zero

Un modello profondo e complesso come una MobileNet (28 layers, 4.2M parametri) non può essere efficacemente addestrato da zero (ovvero a partire da pesi casuali) utilizzando poche decine di migliaia di immagini.

Infatti, specialmente per i livelli molto lontani dall'output che fungono come *feature extractors* iniziali, sono necessarie molte epoche su dataset con centinaia di migliaia di esempi.

Possiamo però partire da un modello preaddestrato su ImageNet (caricandone i pesi) e modificandoli in base al training supervisionato del nostro problema. Questo approccio è noto come **Fine Tuning**.

Bisogna però fare attenzione al fatto che il grafo originale del modello deve essere modificato, rimpiazzando il livello di output (*logit layer*) che originariamente aveva 1001 classi (le 1000 di ImageNet + 1 per la classe sconosciuta), in un nuovo livello di output con tanti neuroni quante sono le nuove classi. Solo per questo nuovo livello i pesi dovranno essere reinizializzati in modo casuale.

# Creazione del grafo

Vediamo ora come creare il grafo del modello e adattarlo per il Fine Tuning, con il supporto dell'API funzionale offerta da Keras.

Nella seguente cella viene caricata la rete MobileNet con i pesi preaddestrati sul dataset ImageNet, viene definita la *loss* e l'ottimizzatore da utilizzare così come le principali metriche di classificazione. Si noti che la rete viene creata con il parametro *include\_top* impostato a *False*. Questo significa che viene restituito un modello privato dell'ultimo layer della rete (che altrimenti presenterebbe 1000+1 output, pari al numero di classi in ImageNet).

Il layer finale viene creato e agganciato nelle righe di codice immediatamente successive in modo da utilizzare un layer con tanti neuroni quante sono le classi del problema da risolvere.

Infine vengono definite le funzioni che verranno utilizzate per svolgere i passi di addestramento e test.

In [ ]:
mobilenet_version = 'small'  # 'small' 'default'

if mobilenet_version == 'default':
    image_side = 224
    width_multiplier = 1.0
elif mobilenet_version == 'small':
    image_side = 128
    width_multiplier = 0.5
else:
    raise ValueError('Versione di MobileNet non supportata.')

# Definizione della shape dei tensori delle immagini
#con 3 canali di colori 128x128x3
IMG_SHAPE = (image_side, image_side, 3)

# Creazione della rete
base_model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE,
                                             alpha=width_multiplier,
                                             #include_top = False non tiene l'ultimo layer originale da 1000 classi
                                             #a noi ci serve un layer da 2 neuroni (nel lab) e da 12 nella competizione
                                             include_top=False,
                                             pooling='avg',
                                             weights='imagenet')

# Se trainable è uguale a False i pesi della rete sono congelati
# In questo caso vogliamo permettere l'addestramento dei pesi
base_model.trainable = True

# Definisce la loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Definisce l'ottimizzatore
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0025)

# Le metriche offerte da Keras accumulano i valori su più batch
# Loss media sul training set
train_loss = tf.keras.metrics.Mean(name='train_loss')  
# Accuratezza sul training set
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')  

# Loss media sul validation/test set
val_loss = tf.keras.metrics.Mean(name='val_loss')
# Accuratezza sul validation/test set
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

# Adatta la rete usando Functional API di Keras
#placeholder (finto layer) che fa da input 
inputs = tf.keras.Input(shape=IMG_SHAPE)
#placeholder per l'output (feature -> output del modello nn ancora passato dal layer di classificazione)
feature = base_model(inputs)
#in questo caso due ma saranno 12 poi
dense_layer = tf.keras.layers.Dense(class_num)
#due logits prima 12 poi
logits = dense_layer(feature)
softmax_layer = tf.keras.layers.Softmax()
probs = softmax_layer(logits)
#creo effettivamente il modello sà che deve prendere gli input e output di una certa shape
model = tf.keras.Model(inputs=inputs, outputs=[logits, probs])

# NOTA: non viene chiamata la funzione .compile di Keras in
# quanto eseguiamo il training a più basso livello senza 
# utilizzare la funzione .fit

# Funzione di train su un minibatch (iterazione)
#i decoratori dicono a tensorflow che deve crearsi dei grafici ottimizzati
#questo lo fa solo la prima volta, dalla seconda volta in poi grazie a questo grafo è molto più efficente e veloce
#una volta ottimizzato però non è facile da debuggare perchè il codice ottimizzato non sarà più visibile
#perchè il codice sarà più "normale" ma trasformato in un grafo
@tf.function
def train_step(images, labels):
    #comincia a tenerti questo tape che tiene traccia del gradiente
    with tf.GradientTape() as tape:
        logits, _ = model(images, training=True)
        loss = loss_object(labels, logits)
    #backpropagation del gradiente
    gradients = tape.gradient(loss, model.trainable_variables)
    #ottimizzatore applica il gradiente per aggiornare i pesi
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # Le metriche accumulano i valori (fino a chiamata di reset)
    train_loss(loss)
    train_accuracy(labels, logits)

# Funzione di inference su un minibatch    
#parte dalle x (images) e dalle y (labels)
#traing=False, stiamo facendo inference quindi non train

@tf.function
def test_step(images, labels):
    logits, probs = model(images, training=False)
    v_loss = loss_object(labels, logits)
    # Le metriche accumulano i valori (fino a chiamata di reset)
    val_loss(v_loss)
    val_accuracy(labels, logits) 
    return probs

5586944/5577668 [==============================] - 0s 0us/step


È possibile visualizzare il modello risultante delle operazioni di caricamento e modifica della rete utilizzando il metodo **.summary()**.

In [ ]:
# base_model.summary()  # Modello prima delle modifiche

model.summary()
#tf.keras.utils.plot_model(model, 'mymodel.png', show_shapes=True) # Richiede pydot e graphviz, opzionale

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 mobilenet_0.50_128 (Functio  (None, 512)              829536    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
 softmax (Softmax)           (None, 2)                 0         
                                                                 
Total params: 830,562
Trainable params: 819,618
Non-trainable params: 10,944
_________________________________________________________________
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/

# Resizing e Suddivisione in Training e Validation Set

Ora che è stato definito il modello da utilizzare è possibile processare il dataset per portare le immagini alla dimensione desiderata (*image\_side*) e per ottenere un validation set separato. La cella seguente mostra come suddividere il dataset in training e validation set utilizzando [**train_test_split**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) mentre il ridimensionamento delle immagini è affidato alla funzione *resize\_images* già vista nelle precedenti esercitazioni.

In [ ]:
print('Resizing in corso ...')
start = time.time()
resized_image_x = ml_utilities.resize_images(raw_images_x, image_side, image_side, cache=memory)
print('Resizing completato in %.2f s.' % (time.time() - start))

train_x, validation_x, train_y, validation_y = train_test_split(resized_image_x, label_y,
                                                                test_size=validation_size,
                                                                #con stratify cercherà di fare lo split in modo da conservare lo split
                                                                #delle classi (il bilanciamento delle classi RICORDATI CHE CI SONO PIù GATTI CHE CANI)
                                                                #stratify è importante su un dataset piccolino
                                                                stratify=label_y,
                                                                random_state=1234)

print('Numero di pattern nel training set:', len(train_x))
print('Numero di pattern nel validation set:', len(validation_x))

Resizing in corso ...
Resizing completato in 4.94 s.
Numero di pattern nel training set: 450
Numero di pattern nel validation set: 200


# Creazione del Dataset TF 2.x

TensorFlow mette a disposizione la classe [**Dataset**](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) in grado di semplificare diverse operazioni di caricamento e manipolazione del dataset. Uno dei metodi più semplici per create un Dataset di TensorFlow è quello di utilizzare la funzione [**from\_tensor\_slices**](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) che trasforma dei tensori NumPy in un Dataset dal medesimo contenuto.

In [ ]:
training_set = tf.data.Dataset.from_tensor_slices((train_x, train_y))
validation_set = tf.data.Dataset.from_tensor_slices((validation_x, validation_y))

## Suddivisione in minibatch

Una volta applicate le procedure di caricamento, di augmentation e finalizzata la suddivisione di training e validation set è possibile, grazie alle funzionalità della classe Dataset, definire la suddivisione in minibatch.

La cella seguente mostra come utilizzare il metodo **.batch(...)** per ottenere un dataset che resituirà un numero di pattern pari a *minibatch\_size* a ogni iterazione. I Dataset di TensorFlow infatti, se non configurati diversamente, restituiscono singoli pattern.

I dataset *train\_dataset* e *valid\_dataset* creati nella seguente cella resituiranno a ogni iterazione un numero di pattern pari a *minibatch\_size*. Le immagini contenute in ogni minibatch verranno unite in un unico tensore di dimensione *(minibatch\_size, image\_side, image\_side, 3)*.

In [ ]:
minibatch_size = 50

#se rimane un minibatch con meno di 50 pattern lo droppa, durante il training può dar fastidio
#cercare di non fare delle frazioni. In questo caso con 450 elementi e 200 elementi nei due set di dati, non ci sono frazioni
train_dataset = training_set.batch(minibatch_size, drop_remainder=True) 
valid_dataset = validation_set.batch(minibatch_size, drop_remainder=True)

# Addestramento
Ora siamo pronti per il *training*. La cella seguente esegue l'addestramento della MobileNet per un numero di epoche pari a **n_epochs** utilizzando minibatch di dimensione **minibatch_size**.

In [ ]:
# Attenzione: eseguire questa cella più volte significa riprendere
# l'addestramento da dove era stato lasciato, cioè non ricomincia
# ogni volta dai pesi iniziali!
n_epochs = 2
   
epochs_training_loss = []
epochs_validation_accuracy = []
epochs_training_accuracy = []

t_start = time.time()
print("Start Training")

total_train_patterns = 0

for epoch in range(n_epochs):

    #vogliamo calcolare le epoche per ogni epoca corrente
    #resetta le metriche all'inizio
    train_loss.reset_states()
    train_accuracy.reset_states()
    val_loss.reset_states()
    val_accuracy.reset_states()
    
    print("Epoch", epoch + 1, " ", end="")
    
    # Train su tutti i minibatch dell'epoca
    #traina su quei 50 pattern ogni volta
    for X_minibatch, y_minibatch in train_dataset:
        total_train_patterns += len(X_minibatch)
        print(".", end="", flush = True)
        train_step(X_minibatch, y_minibatch)  
    
    # Evaluation
    for X_minibatch, y_minibatch in valid_dataset:
        print("+", end="", flush = True)
        test_step(X_minibatch, y_minibatch)
        
    epochs_training_loss.append(train_loss.result().numpy())
    epochs_training_accuracy.append(train_accuracy.result().numpy() * 100)
    epochs_validation_accuracy.append(val_accuracy.result().numpy() * 100)

    print("  Train loss: %.4f  Train acc: %.2f %%  Validation acc: %.2f %%" % (train_loss.result(), train_accuracy.result() * 100, val_accuracy.result() * 100))

print('Salvataggio del modello...')
# Salva tutto: pesi, ottimizzatore, ecc.
save_path = str(Path(checkpoints_path) / ("dogcat_model_%s_with_%d_classes" % (db_name, class_num)))

# .save() può restituire dei warning relativi alla deprecazione di alcune componenti
# interne di TensorFlow. Si tratta di un bug già sistemato nelle versioni nightly
# e che scomparirà nelle prossime versioni.
model.save(save_path)

t_elapsed = time.time()-t_start
print (' -> %d patterns (%.2f sec.) -> %.2f patt/sec' % (total_train_patterns, t_elapsed, total_train_patterns / t_elapsed))

ml_visualization.plot_performance_curves(epochs_training_loss, epochs_training_accuracy, epochs_validation_accuracy)

Start Training
Epoch 1  .

La cella seguente mostra come costruire la *Confusion Matrix* per il validation set. Il codice della cella seguente mostra come eseguire un'epoca sul validation set memorizzandone i risultati. I risultati vengono successivamente utilizzati per disegnare la matrice.

In [ ]:
first_minibatch = True
for X_minibatch, y_minibatch in valid_dataset:
    minibatch_probs = test_step(X_minibatch, y_minibatch)
    if first_minibatch:
        valid_probs = minibatch_probs.numpy()
        first_minibatch = False
    else:
        valid_probs = np.concatenate((valid_probs, minibatch_probs))

# argmax permette di ottenere l'etichetta di classe con 
# punteggio più alto per tutti i pattern
predicted_y = valid_probs.argmax(axis=1)
ml_visualization.plot_confusion_matrix(validation_y, predicted_y, label_names, figsize=(10,8))


# Tuning degli iperparametri

Per il solo problema binario si provi a intervenire sugli iperparametri nel seguente ordine:
- aumentare/diminuire il numero di epoche (**n_epochs**) per garantire convergenza evitando al tempo stesso overfitting;
- tarare il *learning rate* (**learning_rate** nella sezione [Creazione del grafo](#Creazione-del-grafo));
- provare un diverso Optimizer (es. [**Adam**](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) nella sezione [Creazione del grafo](#Creazione-del-grafo));


## Salvataggio del modello migliore
Si consiglia di modificare il codice della cella precedente per poter salvare ad ogni epoca il modello ottenuto (solamente se l'accuratezza di validation è maggiore della massima incontrata fino a quel momento). In questo modo alla fine della fase di *train* il modello salvato sarà il migliore anche se avete impostato un numero di epoche maggiore di quello ottimale.

NOTA: il modello viene salvato con un nome diverso a seconda del tipo di problema affrontato (binario, multiclasse).

## Test

Per il problema binario sono fornite le etichette di classe per il test set. la seguente cella misura l'accuratezza della miglior soluzione ottenuta (utilizzando il modello salvato al passo precedente) sul dataset di test per verificarne l'effettiva capacità di generalizzazione.

In [ ]:
minibatch_size_test = 50

if not test_has_labels:
    # Non sono fornite etichette per il problema di classificazione sui singoli animali
    raise ValueError('Il test set per il problema scelto non ha etichette di classe')
    
raw_test_images_x, test_y = ml_utilities.load_labeled_dataset(test_filelist, db_path, cache=memory)
resized_test_image_x = ml_utilities.resize_images(raw_test_images_x, image_side, image_side, cache=memory)

test_dataset = tf.data.Dataset.from_tensor_slices((resized_test_image_x, test_y))
test_dataset = test_dataset.batch(minibatch_size_test, drop_remainder=False)

# Carica il modello salvato in precedenza su disco.
# Attenzione: se questa istruzione restituisce un errore
# è possibile commentarla e utilizzare il modello
# già memorizzato nella variabile "model". In alternativa
# è possibile aggiornare a TensorFlow 2.2.0 o superiore.
model = tf.keras.models.load_model(save_path)

print("Computing Accuracy on the Test Set ", end="" )
t_start = time.time()

# Riutilizziamo gli oggetti metrics usati per il validation set
val_loss.reset_states()
val_accuracy.reset_states()

first_minibatch = True
for X_minibatch, y_minibatch in test_dataset:
    minibatch_probs = test_step(X_minibatch, y_minibatch)
    if first_minibatch:
        test_probs = minibatch_probs.numpy()
        first_minibatch = False
    else:
        test_probs = np.concatenate((test_probs, minibatch_probs))
        
print(" %.2f %%" % (100 * val_accuracy.result()))
t_elapsed = time.time()-t_start
print (' -> %d patterns (%.2f sec.) -> %.2f patt/sec' % (len(test_y), t_elapsed, len(test_y) / t_elapsed))

predicted_y = test_probs.argmax(axis=1)
print(predicted_y.shape, test_y.shape)
ml_visualization.plot_confusion_matrix(test_y, predicted_y, label_names, figsize=(10,8))

## Visualizzazione errori
La cella seguente permette di visualizzare le immagini di test che la rete classifica erroneamente. Sopra ad ogni immagine è riportato il nome della classe corretta mentre a lato le classi più probabili.

In [ ]:
errors = predicted_y != test_y
error_idx = np.where(errors == True)[0]

# Visualizzazione immagini
image_per_row = min(3, len(error_idx))
top_class_count = 3
max_images = min(30, len(error_idx))

row_count = math.ceil(max_images / image_per_row)
column_count = image_per_row
plt.rcParams.update({'font.size': 12})
_, axs = plt.subplots(row_count, column_count, figsize=(15, 4*row_count), squeeze=False)

for i in range(row_count):
    for j in range(column_count):
        axs[i,j].axis('off')

for i in range(max_images):
    q = i//image_per_row
    r = i%image_per_row
    idx = error_idx[i]
    axs[q,r].imshow(resized_test_image_x[idx])
    axs[q,r].set_title(label_names[test_y[idx]])
    
    sorted_test_probs, sorted_label_names= (list(t) for t in zip(*sorted(zip(test_probs[idx], label_names))))
    text=''
    for j in range(min(top_class_count,class_num)):
        text+='{}: {:.2f}\n'.format(sorted_label_names[-j-1],sorted_test_probs[-j-1])
    
    axs[q,r].text(resized_test_image_x[idx].shape[1]*1.05, resized_test_image_x[idx].shape[0]*0.5, text, horizontalalignment='left', verticalalignment='center')
plt.show()